# Data Generation
Assumptions that we made:
- `TCS_Revenue` & `Client_Revenue` are generated using a mix of distrubutions like Normal, Unifrom etc.
- Assumed some probabilty from one stage to another transit stage
- Probability that an opportunity converts (moves from current stage to next stage) is proportional to the negative exponential of the time spent in that stage


In [61]:
# Importing libraries
import numpy as np
import random
import matplotlib
from matplotlib import pyplot as plt
import datetime
from datetime import datetime as dt
from scipy import stats
import pandas as pd
import random
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Distributing the data points
def weighted_pick(weights, n_picks):
    t = np.cumsum(weights)
    s = np.sum(weights)
    return np.searchsorted(t, np.random.rand(n_picks)*s)


# Pre Stages
pre_stages = [('Stage 1', 0.75, 20), ('Stage 2', 0.6, 20),('Stage 3', 0.5, 20), ('Stage 4', 0.4, 20), ('Stage 5', 0.3, 20), ('Stage 6', 0.2, 20), ('Stage 7', 0.25, 20), ('Stage 8', 0.15, 20)]

# Closed Stages
closed_stages = ['Stage 9', 'Stage 10', 'Stage 11', 'Stage 12', 'Stage 13']

#Success Stages
success_stages = ['Stage 9']

# Opportunity Client Attributes
df_client = pd.DataFrame(columns=['Name','Client_Revenue', 'Region','Client_Domain','Existing_Customer'])

# Total Number of Opportunities
NUM_POINTS = 1146

# Mean of Normal Distribution
Average_Revenue = [50000,100000,150000,200000,250000]

# Standard Deviation of Normal Distribution
SD_Revenue = [5000,10000,15000, 20000, 25000]

# Client Domains List (source: https://infotechlead.com/wp-content/uploads/2013/10/TCS-second-quarter-revenue.png)
client_domains=['BFSI',
               'Telecom',
               'Retail & Distribution',
               'Manufacturing',
               'Hi-Tech',
               'Healthcare',
               'Travel and Hospitality',
               'Energy & Utilities',
               'Media',
               'Others']
# Region List (source: https://image.slidesharecdn.com/tcspptfinal-151119185457-lva1-app6892/95/tcs-ppt-final-12-638.jpg?cb=1447959470)
regions = ['north america', 'middle east and africa', 'asia-pacific','india','europe','UK','latin america','others']

# Reading Client Names
clients = open('client_new.txt', 'r',encoding='utf-8').read().splitlines()

# Client Revenue Generation
sales_opps1 = [(entry.title(), np.random.choice([
    np.random.normal(np.random.choice(Average_Revenue,p=[0.20, 0.20, 0.2, 0.2, 0.2]), 
                     np.random.choice(SD_Revenue,p=[0.20, 0.20, 0.2, 0.2, 0.2])),
    np.random.uniform(40000,275000),
    np.random.randint(30000,150000)],
    p = [0.4,0.4,0.2])) for entry in np.random.choice(clients, NUM_POINTS, replace=False)]

names=[]
revenues=[]

for name,revenue in sales_opps1:
    names.append(name)
    revenues.append(revenue)
    
# Distributing Client Domain to every opportunity
domains = np.random.choice(client_domains, NUM_POINTS, replace=True,p=[0.431,0.093,0.139,0.084,0.054,0.057,0.034,0.038,0.022,0.048])

# Distributing Region to every opportunity
client_regions = np.random.choice(regions, NUM_POINTS, replace=True,p=[0.536,0.077,0.077,0.085,0.098,0.0152,0.031,0.0808000000000001])

# Generating Existing customer parameter
existing_or_no = np.random.choice([1,0], NUM_POINTS, replace=True,p=[0.7,0.3])

# Assigning generated parameters to Dataframe
df_client['Name'] = names
df_client['Client_Revenue']=revenues
df_client['Client_Domain']=domains
df_client['Region']=client_regions
df_client['Existing_Customer']=existing_or_no

# Opportunity BU Attributes
df_BU = pd.DataFrame(columns=['Name','TCS_Revenue', 'TCS_Domain'])

# Mean of Normal Distribution
Average_TCS_Revenue = [5000,10000,15000,20000,25000]

# Standard Deviation of Normal Distribution
SD_TCS_Revenue = [500,1000,1500,2000,2500]

# BU's Domains List
BUs=['Application development and maintenance',
'Asset leverage solutions',
'Assurance services',
'Business process outsourcing',
'Consulting',
'Engineering and Industrial services',
'Enterprise solution',
'IT infrastructure services',
'Cognitive Business Operations',
'Cloud Infrastructure',
'Automation and AI']

sales_opps2 = [(entry.title(), np.random.choice([
    np.random.normal(np.random.choice(Average_TCS_Revenue,p=[0.20, 0.20, 0.2, 0.2, 0.2]), 
                     np.random.choice(SD_TCS_Revenue,p=[0.20, 0.20, 0.2, 0.2, 0.2])),
    np.random.uniform(4000,27500),
    np.random.randint(3000,15000)],
    p = [0.4,0.3,0.3])) for entry in np.random.choice(clients, NUM_POINTS, replace=False)]

names=[]
revenues=[]

for name,revenue in sales_opps2:
    names.append(name)
    revenues.append(revenue)
    
# Distributing TCS Domain to every opportunity
domains = np.random.choice(BUs, NUM_POINTS, replace=True,p=[0.4,0.05,0.05,0.06,0.04,0.05,0.05,0.1,0.1,0.05,0.05])

# Assigning generated parameters to Dataframe
df_BU['Name'] = names
df_BU['TCS_Revenue']=revenues
df_BU['TCS_Domain']=domains

# Merging Client and BUs Dataframe into one dataframe
new_df = pd.merge(df_client,df_BU,on = 'Name')

- Plotting distribution of `Client_Revenue` & `TCS_Revenue`

In [62]:
import cufflinks as cf
import numpy as np
import plotly
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.offline import init_notebook_mode
import plotly.express as px

@interact
def scatter_plot(x=list(new_df.select_dtypes('number').columns)):

    fig = px.histogram(new_df, x=x)
    fig.show()

interactive(children=(Dropdown(description='x', options=('Client_Revenue', 'Existing_Customer', 'TCS_Revenue')…

- So, for a given stage in the data pipeline, for each opportunity left in the previous stage, for every day between when the opportunity entered the stage and now we create a multinomial line. 

- The multinomial line is going to return a one of the three outputs (x1,x2,x3)
- In 99% cases it will give output as x1 that means, for every day between the opportunity entering the state and today there’s a 99% the opportunity will still be in that state at the end of the day. 
- If output is X2, means that the opportunity succeeded on that particular day (with probability given by the stage parameter)
- If output is X3, then the opportunity died on that particular day. 

In [ ]:
# Converting dataframe to a list of tuple
sales_opportunities = [tuple(x) for x in new_df.to_records(index=False)]
sales_opportunities

# Taking a Timeline of 3 Years
start_date = datetime.datetime.now() - datetime.timedelta(days = int(1095))
days_range = range(365*3)
y = [float(entry)/365. for entry in days_range]

# Distributing the opportunities
indices = weighted_pick(np.exp(y), NUM_POINTS)

# Intialize every opportunity with Stage 1
sales_data = [[pre_stages[0][0],name_value_pair[0], name_value_pair[1], name_value_pair[2],name_value_pair[3],name_value_pair[4],name_value_pair[5],name_value_pair[6], start_date + datetime.timedelta(days = int(index)),] for name_value_pair, index in zip(sales_opportunities, indices)]

# Creating new dataframe remaining_opportunities_frame and sales_data_frame
remaining_opportunities_frame = pd.DataFrame(sales_data)
remaining_opportunities_frame.columns = ['Stage', 'Name', 'Client_Revenue', 'Region','Client_Domain','Existing_Customer', 'TCS_Revenue', 'TCS_Domain','TimeStamp']
sales_data_frame = pd.DataFrame(sales_data)
sales_data_frame.columns = ['Stage', 'Name', 'Client_Revenue', 'Region','Client_Domain','Existing_Customer', 'TCS_Revenue', 'TCS_Domain','TimeStamp']
sales_data_frame.head()

# Adding trasition stages in sales_data_frame
finished_list = set([])
idx = 0
for stage_index, stage in enumerate(pre_stages[1:]):
    idx = idx + 1
    next_stage = pd.DataFrame([(sales_opp[1], index, np.argmax(entry)) 
                               for sales_opp in sales_data 
                               for index, entry in enumerate(
                                   np.random.multinomial(1, [0.99, (1. - stage[1])/100., 
                                                             stage[1]/100.0], 
                                                         (datetime.datetime.now() - sales_opp[8]).days)) 
                               if entry[0] != 1 and sales_opp[1] not in finished_list])

    next_stage.columns = ['Name', 'TimeStamp', 'Status']
    meh = next_stage.iloc[next_stage.groupby('Name').TimeStamp.idxmin()]
    tempy_frame = meh.merge(remaining_opportunities_frame[['Name','Client_Revenue', 'Region','Client_Domain','Existing_Customer', 'TCS_Revenue', 'TCS_Domain','TimeStamp',]], how='inner', on='Name')
    tempy_frame['new_date'] = tempy_frame.apply(lambda x: sales_data_frame[sales_data_frame['Name']==x.Name]['TimeStamp'].tolist()[-1] + datetime.timedelta(days = x.TimeStamp_x), axis=1)
    tempy_frame = tempy_frame[['Name', 'Status', 'Client_Revenue', 'Region','Client_Domain',
       'Existing_Customer','TCS_Revenue', 'TCS_Domain','new_date']]
    tempy_frame.columns = ['Name', 'Status', 'Client_Revenue', 'Region','Client_Domain',
       'Existing_Customer','TCS_Revenue', 'TCS_Domain', 'TimeStamp']

    success_frame = tempy_frame[tempy_frame.Status == 1]
    success_frame = success_frame.drop('Status', 1)
    success_frame.insert(0, 'Stage', pre_stages[stage_index + 1][0] if stage_index + 1 < len(pre_stages) else success_stages[0])

    failure_frame = tempy_frame[tempy_frame.Status == 2]
    failure_frame = failure_frame.drop('Status', 1)
    failure_frame.insert(0, 'Stage', np.random.choice(closed_stages, p=[0.3,0.2,0.2,0.2,0.1]))
    sales_data_frame = pd.concat([sales_data_frame,failure_frame,success_frame])

    finished_frame = sales_data_frame.groupby('Name').apply(lambda x: x.Stage.isin(closed_stages).any())
    finished_list = set(finished_list).union(set(finished_frame[finished_frame == True].index.values))
    remaining_opportunities = remaining_opportunities_frame[~remaining_opportunities_frame.Name.isin(finished_list)]

In [64]:
# Converting into DateTime format and sorting dataframe by Name & TimeStamp 
sales_data_frame['TimeStamp'] = pd.to_datetime(sales_data_frame['TimeStamp'],format='%d-%m-%Y %H:%M')
sales_data_frame= sales_data_frame.sort_values(by = ['Name','TimeStamp'])

# Assigning ID to every opportunity
uniq = sales_data_frame['Name'].unique()
uniq_id=[]
id_=0
for i in uniq:
    for j in sales_data_frame['Name']:
        if i==j:
            uniq_id.append(id_)
    id_+=1
sales_data_frame['ID'] = uniq_id

# Downloading the dataset
# sales_data_frame.to_csv('experimental_data.csv', sep=',', index=False,header=True)

In [65]:
import datetime
df = sales_data_frame.copy()
my_df  = pd.DataFrame()
arr = df.Name.unique()
for x in arr:
    new_df = df[df.Name == x]
    new_df = new_df.reset_index(drop = 1)
    k = list(new_df['Stage']) 
    new_df["NewTimeStamp"] = ""
    z = 0
    for i in range(0,len(k)):
        end_date = new_df['TimeStamp'][0] + datetime.timedelta(days=z)
        new_df.loc[i,'NewTimeStamp']= end_date
        z = z + 20
    my_df = my_df.append(new_df)

my_df['TimeStamp'] = my_df['NewTimeStamp']
my_df.drop(['NewTimeStamp'],axis = 1, inplace = True)
# Saving the dataset
my_df.to_csv('exp_2.csv', sep=',', index=False,header=True)